### 스마트 냉장고
* 데이터 셋 : 과채류 30 종류
* 피쳐/속성 : 그림 파일
* 타겟/라벨 : 폴더명
* 학습-방법 : 지도학습 > 분류 > 다중분류
* 알고리즘 : 
* 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비
<hr>

In [1]:
# ---------------------------------------------------------------------
# 모델링 관련 모듈 로딩
# ---------------------------------------------------------------------
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchinfo import summary
from torch.utils.data import random_split

from torchmetrics.classification import accuracy
from torchmetrics.classification import MulticlassF1Score, MulticlassAccuracy, MulticlassConfusionMatrix
import torch.optim.lr_scheduler as lr_scheduler

import torchvision.models as models

# ---------------------------------------------------------------------
# 데이터 분석 관련 모듈 로딩
# ---------------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------------------------------------------------
# 이미지 관련 모듈 로딩
# ---------------------------------------------------------------------
import cv2
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2

# ---------------------------------------------------------------------
# 기타 모듈 로딩
# ---------------------------------------------------------------------
import time
import os

# 활용 패키지 버전 체크
print(f'torch Ver.:{torch.__version__}')
print(f'pandas Ver.:{pd.__version__}')
print(f'numpy Ver.:{np.__version__}')

torch Ver.:2.4.1
pandas Ver.:2.0.3
numpy Ver.:1.24.3


In [2]:
# DEVICE 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

* 이미지 데이터셋 생성
<hr>

In [3]:
transConvert = transforms.Compose([transforms.Resize([236]), transforms.CenterCrop(224),transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [4]:
IMG_PATH = r'C:\Users\KDP-14\Desktop\VSCode\LocalData\self_project_image\fruit_veget_image'

In [5]:
# 이미지 데이터셋 생성
imgDS = ImageFolder(root=IMG_PATH, transform=transConvert)

In [6]:
type(imgDS)

torchvision.datasets.folder.ImageFolder

In [7]:
print(f'imgDS.classes       : {imgDS.classes}')
print(f'imgDS.class_to_idx  : {imgDS.class_to_idx}')
print(f'imgDS.targets       : {imgDS.targets}')
for img in imgDS.imgs:
    print(f'imgDS.imgs       : {img}')
    break

imgDS.classes       : ['apple', 'banana', 'beetroot', 'bell_pepper', 'cabbage', 'carrot', 'cauliflower', 'chilli_pepper', 'corn', 'cucumber', 'eggplant', 'garlic', 'ginger', 'grapes', 'kiwi', 'lemon', 'lettuce', 'mango', 'onion', 'orange', 'pear', 'peas', 'pineapple', 'pomegranate', 'potato', 'radish', 'spinach', 'sweetpotato', 'tomato', 'watermelon']
imgDS.class_to_idx  : {'apple': 0, 'banana': 1, 'beetroot': 2, 'bell_pepper': 3, 'cabbage': 4, 'carrot': 5, 'cauliflower': 6, 'chilli_pepper': 7, 'corn': 8, 'cucumber': 9, 'eggplant': 10, 'garlic': 11, 'ginger': 12, 'grapes': 13, 'kiwi': 14, 'lemon': 15, 'lettuce': 16, 'mango': 17, 'onion': 18, 'orange': 19, 'pear': 20, 'peas': 21, 'pineapple': 22, 'pomegranate': 23, 'potato': 24, 'radish': 25, 'spinach': 26, 'sweetpotato': 27, 'tomato': 28, 'watermelon': 29}
imgDS.targets       : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [10]:
len(imgDS)

33367

In [ ]:
## train, test 분리
trainDS, validDS, testDS = random_split(imgDS, [11717,3905,3905])

trainDL = DataLoader(trainDS, batch_size=32, shuffle=True)
validDL = DataLoader(validDS, batch_size=32, shuffle=True)
testDL = DataLoader(testDS, batch_size=32, shuffle=True)

In [ ]:
convnext_tiny = models.convnext_tiny(pretrained = True, weights = models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1)

In [ ]:
### 데이터 로딩
class Custommodel(nn.Module):
    def __init__(self):
        super(Custommodel, self).__init__()
        self.features = convnext_tiny.features
        self.avgpool = convnext_tiny.avgpool
        self.classifier = convnext_tiny.classifier
        self.fc = nn.Sequential(
                nn.ReLU(),
                nn.Linear(1000, 500),
                nn.Dropout(0.25),
                nn.ReLU(),
                nn.Linear(500, 250),
                nn.Dropout(0.25),
                nn.ReLU(),
                nn.Linear(250, 1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        x = self.fc(x) 
    
        return F.sigmoid(x)